In [1]:
import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
import pandas as pd

import json

import json
from datetime import datetime
from pymongo import MongoClient, GEOSPHERE
from pymongo.errors import (PyMongoError, BulkWriteError)

import shapely
import numpy as np

import pymongo

In [2]:
def mongo(url, db, table):
    mongo_client = pymongo.MongoClient(f"mongodb://{url}:27017/")
    mongo_db = mongo_client[db]
    mongo_tbl = mongo_db[table]
    return mongo_tbl

In [3]:
inputfile = "berths_gdf.geojson"
to_collection = "berths"
to_database = "rail"
to_server = ''
to_port = '27017'
db_user = None

In [4]:
with open(inputfile,'r') as f:
  geojson = json.loads(f.read())
  
for feature in geojson['features']:
    feature['id'] = feature['properties']['berth_id']
    feature['_id'] = feature['properties']['berth_id']

In [5]:
len(geojson['features'])

17434

In [6]:
berths = pd.read_csv('berths.csv')


In [7]:
ids = [x['id'] for x in geojson['features']]

for index, row in berths.iterrows():
    if row['berth_id'] not in ids:
        new_row = {
            'type': 'Feature',
            'properties':  {'index': row['idx'],
                            'berth_id': row['berth_id'],
                            'berth': row['berth'],
                            'area_id': row['area_id'],
                            'NoMatches': '0'},
            'id': row['berth_id'],
            '_id': row['berth_id']}
        geojson['features'].append(new_row)


In [8]:
uri = 'mongodb://' + to_server + ':' + to_port +'/'

client = MongoClient(uri)
db = client[to_database]
collection = db[to_collection]

# create 2dsphere index and initialize unordered bulk insert
collection.create_index([("geometry", GEOSPHERE)])


# for feature in geojson['features']:

collection.insert_many(geojson['features'])


In [9]:
def get_berth_geo(berth, mongo):
    try:
        response = mongo.find({'_id': berth}, {"_id": 0, "geometry": 1})
        responses = list(response)
        return responses 
    except Exception as e: 
        print("Exception - Error calling mongodb - get item in tt table")
        print(f"Mongo Exception - Function: get_berth_geo - ID: {berth} - {e}")
        return None



In [12]:
berth = "A2_0598"
mongo_berth = mongo(to_server, 'rail', 'berths')
x = get_berth_geo(berth, mongo_berth)

In [11]:
gdf = gpd.GeoDataFrame(geojson['features'])

In [13]:
sigs = ["Y8_0600",
"Y8_0603",
"Y8_0602",
"Y8_0605",
"Y8_0607",
"Y8_0604",
"Y8_0606",
"Y8_0608",
"Y8_0609",
"Y8_0617",
"Y8_0619",
"Y8_0620",
"Y8_0622",
"Y8_0623",
"Y8_0625",
"Y8_0624",
"Y8_0627",
"Y8_0626",
"Y8_0628",
"Y8_0629",
"Y8_0630",
"Y8_0632",
"Y8_0633",
"Y8_0635",
"Y8_0637",
"Y8_0639",
"Y8_0634",
"Y8_0636",
"Y8_0638",
"Y8_0640",
"Y8_0651",
"Y8_0653",
"Y8_0650",
"Y8_0652",
]

In [26]:
geojson['features'][0]

{'id': 'A2_0595',
 'type': 'Feature',
 'properties': {'berth_id': 'A2_0595',
  'berth': '0595',
  'area_id': 'A2',
  'marker-color': '#7BA832',
  'ASSETID': '9001049997'},
 'geometry': {'type': 'Point',
  'coordinates': [0.5522817644904565, 51.17139677242064]},
 '_id': 'A2_0595'}

In [27]:
x = []
for t in geojson['features']:
    if t['id'] in sigs:
        x.append(t)

In [31]:
import copy
geojson_welwyn = copy.deepcopy(geojson)
geojson_welwyn['features'] = x

In [34]:
geojson_welwyn

{'type': 'FeatureCollection',
 'features': [{'id': 'Y8_0600',
   'type': 'Feature',
   'properties': {'berth_id': 'Y8_0600',
    'berth': '0600',
    'area_id': 'Y8',
    'marker-color': '#EC4463',
    'ASSETID': '9001015150'},
   'geometry': {'type': 'Point',
    'coordinates': [-0.20436350849326468, 51.79982669333365]},
   '_id': 'Y8_0600'},
  {'id': 'Y8_0602',
   'type': 'Feature',
   'properties': {'berth_id': 'Y8_0602',
    'berth': '0602',
    'area_id': 'Y8',
    'marker-color': '#EC4463',
    'ASSETID': '9001015061'},
   'geometry': {'type': 'Point',
    'coordinates': [-0.20422732606838445, 51.799828710731994]},
   '_id': 'Y8_0602'},
  {'id': 'Y8_0603',
   'type': 'Feature',
   'properties': {'berth_id': 'Y8_0603',
    'berth': '0603',
    'area_id': 'Y8',
    'marker-color': '#EC4463',
    'ASSETID': '9001015150'},
   'geometry': {'type': 'Point',
    'coordinates': [-0.20346611664408615, 51.80171129140349]},
   '_id': 'Y8_0603'},
  {'id': 'Y8_0604',
   'type': 'Feature',
   

In [33]:
import json

with open('welwyn_example.geojson', 'w') as convert_file:
     convert_file.write(json.dumps(geojson_welwyn))

